In [1]:
import at
import at.plot
import sys
if sys.version_info.minor < 9:
    from importlib_resources import files, as_file
else:
    from importlib.resources import files, as_file

In [2]:
from at.future import Variable, VariableList, ElementVariable, match
from at import LocalOpticsObservable, ObservableList

ImportError: cannot import name 'match' from 'at.future' (/Users/laurent/dev/libraries/at/pyat/at/future.py)

# Correlated variables

In this example of correlation between variables, we vary the length of the two drifts
surrounding a monitor but keep the sum of their lengths constant.

Using these 2 correlated variables, we will match a constraint on the monitor.

## Load a test lattice

In [ ]:
fname = 'hmba.mat'
with as_file(files('machine_data') / fname) as path:
    hmba_lattice = at.load_lattice(path)

Isolate the two drifts

In [ ]:
dr1 = hmba_lattice["DR_01"][0]
dr2 = hmba_lattice["DR_02"][0]

Get the total length to be preserved

In [ ]:
l1 = dr1.Length
l2 = dr2.Length
ltot = l1 + l2

Create a constraint {math}`\beta_y=3.0` on `BPM_01`

In [ ]:
obs1 = LocalOpticsObservable('BPM_01', 'beta', plane='v', target=3.0)

## Method 1: using parameters

We parametrise the lengths of the drifts surrounding the monitor

### Parametrise the two drifts:

In [ ]:
param1 = dr1.parametrise('Length')
dr2.Length = ltot-param1

### Run the matching

In [ ]:
variables = VariableList([param1])
constraints = ObservableList(hmba_lattice, [obs1])
match(hmba_lattice, variables, constraints, verbose=1)

### Show the modified lattice

In [ ]:
for elem in hmba_lattice.select([2,3,4]):
    print(elem)

In [ ]:
hmba_lattice.plot_beta()

The first BPM is moved to a location where {math}`\beta_y=3.0`

Restore the lattice

In [ ]:
dr1.Length = l1
dr2.Length = l2

## Method 2: custom variable

We define a new variable class which will act on the two elements and fulfil the constraint

### Define a variable coupling two drift lengths so that their sum is constant:

In [ ]:
class ElementShifter(Variable):
    def __init__(self, dr1, dr2, total_length=None, **kwargs):
        """Varies the length of the elements *dr1* and *dr2*
        keeping the sum of their lengths equal to *total_length*.

        If *total_length* is None, it is set to the initial total length
        """        
        # store indexes of the 2 variable elements
        self.dr1 = dr1
        self.dr2 = dr2
        # store the initial total length
        if total_length is None:
            total_length = dr1.Length + dr2.Length
        self.length = total_length
        super().__init__(bounds=(0.0, total_length), **kwargs)

    def _setfun(self, value, ring=None):
        dr1.Length = value
        dr2.Length = self.length - value

    def _getfun(self, ring=None):
        return  dr1.Length

Create a variable moving the monitor `BPM_01`

In [ ]:
var0 = ElementShifter(dr1, dr2, name='DR_01', total_length=ltot)

### Run the matching

In [ ]:
variables = VariableList([var0])
constraints = ObservableList(hmba_lattice, [obs1])
match(hmba_lattice, variables, constraints, verbose=1)

### Show the modified lattice

In [ ]:
for elem in hmba_lattice.select([2,3,4]):
    print(elem)

In [ ]:
hmba_lattice.plot_beta()